In [ ]:
import torch
from model.LightGCN import *
from preprocess.Yelp import *
from evaluation.LightGCN_evaluation import *
pd.options.display.max_rows = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
path = './dataset/yelp2018'
dataset = Yelp(path)

# Data(num_nodes=144242, edge_index=[2, 2380730], edge_label_index=[2, 603378])
data = dataset.get()
num_users, num_books = dataset.getNumber()
config = {
    'k': 20,
    'lr': 0.001,
    'epochs': 1000,
    'num_layers': 2,
    'batch_size': 8192,
    'embedding_dim': 64,
    'num_users': num_users,
    'num_books': num_books,
    'tuning_type': None,
}
model = LightGCN(
    num_nodes=data.num_nodes,
    embedding_dim=config['embedding_dim'],
    num_layers=config['num_layers'],
).to(device)

In [ ]:
model, epoch_tracks, test_topks = lightgcn_eva(model, config, data, device)
# Store the model parameters
torch.save(model.state_dict(), f"lightGCN_Yelp2018_{config['epochs']}_Epochs_Top_{config['k']}.pt")

In [ ]:
import matplotlib.pyplot as plt
K = config["k"]
plt.plot(epoch_tracks, [precision for precision, _, _ in test_topks],
         label="Test")
plt.ylabel(f"Top {K} precision")
plt.xlabel("Epochs")
plt.legend()
plt.show()

In [ ]:
plt.plot(epoch_tracks, [recall for _, recall, _ in test_topks],
         label="Test")
plt.ylabel(f"Top {K} recall")
plt.xlabel("Epochs")
plt.legend()
plt.show()